In [ ]:
import numpy as np
from gwpy.timeseries import TimeSeries
from scipy import signal
from scipy.fftpack import fft
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy import interpolate

In [ ]:
# load data
data_H1 = np.load("/Users/jkliao117/Desktop/noise time samples H1.npz")['data_H1']
data_L1 = np.load("/Users/jkliao117/Desktop/noise time samples L1.npz")['data_L1']
avg_psd_H1 = np.load("/Users/jkliao117/Desktop/avg psd H1.npz")['avg_psd_H1']
avg_psd_L1 = np.load("/Users/jkliao117/Desktop/avg psd L1.npz")['avg_psd_L1']
psdfreqs = np.load("/Users/jkliao117/Desktop/psd freqs.npz")['psdfreqs']
(sample_size, data_size) = data_H1.shape

In [ ]:
# define frequencies based on the specified data size
fs = 4096
dt = 1/fs
freqs = np.fft.rfftfreq(fs*5, dt)
# interpolate H1 psd 
interp_func_H1 = interpolate.interp1d(psdfreqs,avg_psd_H1)
power_H1 = interp_func_H1(freqs)
interp_func_L1 = interpolate.interp1d(psdfreqs,avg_psd_L1)
power_L1 = interp_func_L1(freqs)

In [ ]:
# noise generation
# n(f)=np.random.normal(sigma=sqrt(P(t))/2)
fnoise_H1 = np.random.normal(0,np.sqrt(power_H1/2))+1.j*np.random.normal(0,np.sqrt(power_H1/2))

In [ ]:
# simulated noise power spectrum
plt.figure(figsize=(10,7.5))
plt.plot(freqs,power_L1,linewidth=1.5,label="Averaged Real LIGO L1 Noise",color='darkcyan')
plt.plot(freqs,power_H1,linewidth=1.5,label="Averaged Real LIGO H1 Noise",color='darkorange')
plt.xscale('log')
plt.yscale('log')
plt.grid(False)
plt.xlabel('Frequency (Hz)',fontsize=25)
plt.ylabel('Power Spectral Density ($Hz^{-1}$)',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.locator_params(tight=True)
plt.legend(loc="lower left",fontsize=20)
plt.savefig('/Users/jkliao117/Desktop/PSD.png', transparent=True)

In [ ]:
# simulated noise power spectrum
plt.figure(figsize=(10,7.5))
plt.plot(freqs,abs(fnoise_H1)**2,linewidth=1.5,label="Simulated Noise")
plt.plot(psdfreqs,avg_psd_H1,linewidth=1.5,label="Averaged Real LIGO H1 Noise")
plt.xscale('log')
plt.yscale('log')
plt.grid(False)
plt.xlabel('Frequency (Hz)',fontsize=25)
plt.ylabel('Power Spectral Density ($Hz^{-1}$)',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.locator_params(tight=True)
plt.legend(loc="lower left",fontsize=20)
plt.savefig('/Users/jkliao117/Desktop/PSD_sim.png', transparent=True)

In [ ]:
# bandpass
# [35,350]Hz window recommended by "A guide to LIGO-Virgo detector noise and extraction of transient gravitational-wave signals"
fs = 4096
f_high_pass = 35
f_low_pass = 350
mask = (freqs > f_high_pass) & (freqs < f_low_pass)
bandp = signal.tukey(np.sum(mask),alpha=0.25)
fnoise_H1_banded = np.zeros_like(fnoise_H1)
fnoise_H1_banded[mask] = fnoise_H1[mask]*bandp

# inverse FFT
tnoise_H1 = (fs**0.5)*np.fft.irfft(fnoise_H1_banded,norm="ortho")
tnoise_H1_size = tnoise_H1.shape[0]
time = np.arange(0,tnoise_H1_size)/fs

In [ ]:
plt.figure(figsize=(10,7.5))
plt.plot(freqs,abs(fnoise_H1)**2,label="Simulated Noise")
plt.plot(freqs[mask],(np.abs(fnoise_H1)**2)[mask],label="Simulated Noise (Bandpassed)",color='salmon')
plt.plot(freqs[mask],(np.abs(fnoise_H1)**2)[mask]*bandp,label="Simulated Noise (Bandpassed and Windowed)",color='mediumseagreen')
plt.xscale('log')
plt.yscale('log')
plt.grid(False)
plt.xlabel('Frequency (Hz)',fontsize=25)
plt.ylabel('Power Spectral Density ($Hz^{-1}$)',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.locator_params(tight=True)
plt.legend(loc="lower left",fontsize=20)
plt.savefig('/Users/jkliao117/Desktop/PSD_sim_bpwd.png', transparent=True)

In [ ]:
# bandpassing real LIGO noise data

# FFT
bandpt = signal.tukey(data_size,alpha=0.25)
fdata_H1 = np.fft.rfft(data_H1*bandpt,norm="ortho")
ffreq_H1 = np.fft.rfftfreq(data_size, d=1/fs)

# bandpass
maskf = (ffreq_H1 > f_high_pass) & (ffreq_H1 < f_low_pass)
bandpf = signal.tukey(np.sum(maskf),alpha=0.25)
fdata_H1_banded = np.zeros_like(fdata_H1)
for i in range(sample_size):
    fdata_H1_banded[i][maskf] = fdata_H1[i][maskf]*bandpf
    
# inverse FFT
tdata_H1_banded = np.fft.irfft(fdata_H1_banded,norm="ortho")
tdata_H1_size = tdata_H1_banded.shape[1]
tdata_H1_time = np.arange(0,tdata_H1_size)/fs

In [ ]:
fig, ax = plt.subplots(figsize=(10,5.5)) 
ax.yaxis.get_offset_text().set_fontsize(15)
plt.plot(time,tnoise_H1,label="Simulated Noise")
plt.grid(False)
plt.xlabel('Time (s)',fontsize=25)
plt.ylabel('Strain',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.locator_params(tight=True)
plt.legend(loc="lower left",fontsize=20)
plt.locator_params(tight=True)
plt.savefig('/Users/jkliao117/Desktop/sim_tnoise.png', transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5)) 
ax.yaxis.get_offset_text().set_fontsize(15)
plt.plot(time,tnoise_H1,label="Simulated Noise")
plt.plot(time[:int(fs*5)],tdata_H1_banded[0][int(fs*5):int(fs*10)],label="Real LIGO H1 Noise (Bandpassed)",alpha=0.75)
plt.grid(False)
plt.xlabel('Time (s)',fontsize=25)
plt.ylabel('Strain',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.locator_params(tight=True)
plt.legend(loc="lower left",fontsize=20)
plt.ylim(-1.25e-21,1.25e-21)
plt.savefig('/Users/jkliao117/Desktop/sim_tnoise_comp1.png', transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5)) 
ax.yaxis.get_offset_text().set_fontsize(15)
plt.plot(time[:int(fs*0.1)],tnoise_H1[:int(fs*0.1)],label="Simulated Noise")
plt.plot(tdata_H1_time[:int(fs*0.1)],tdata_H1_banded[0][int(fs*5):int(fs*5.1)],label="Real LIGO H1 Noise (Bandpassed)")
plt.grid(False)
plt.xlabel('Time (s)',fontsize=25)
plt.ylabel('Strain',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.locator_params(tight=True)
plt.legend(loc="lower left",fontsize=20)
plt.ylim(-1.25e-21,1.25e-21)
plt.savefig('/Users/jkliao117/Desktop/sim_tnoise_comp2.png', transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5)) 
ax.yaxis.get_offset_text().set_fontsize(15)
plt.plot(tdata_H1_time, tdata_H1_banded[0], label = "GW150914")
plt.plot(tdata_H1_time, tdata_H1_banded[1], label = "GW151012")
plt.plot(tdata_H1_time, tdata_H1_banded[2], label = "GW151226")
plt.plot(tdata_H1_time, tdata_H1_banded[3], label = "GW170104")
plt.plot(tdata_H1_time, tdata_H1_banded[4], label = "GW170608")
plt.plot(tdata_H1_time, tdata_H1_banded[5], label = "GW170729")
plt.plot(tdata_H1_time, tdata_H1_banded[6], label = "GW170809")
plt.plot(tdata_H1_time, tdata_H1_banded[7], label = "GW170814")
plt.plot(tdata_H1_time, tdata_H1_banded[8], label = "GW170817")
plt.plot(tdata_H1_time, tdata_H1_banded[9], label = "GW170818")
plt.plot(tdata_H1_time, tdata_H1_banded[10], label = "GW170823")
plt.grid(False)
plt.xlabel('Time (s)',fontsize=20)
plt.xlim(0,20)
plt.ylabel('Strain',fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.locator_params(tight=True)
plt.legend(loc="lower right",fontsize=15)